## Modules

In [4]:
import requests
import xml.etree.ElementTree as et
import bs4
from lxml import html
import pprint
import pandas as pd

## TestCase

1. 참값
2. 논문이 없는 경우
3. 저자 정보가 잘못된 경우 
4. KCI 등재정보가 잘못된 경우
5. 해외 학술지 정보가 잘못된 경우
6. 영향력 지수가 잘못된 경우 
7. KCI에는 등재되었지만 해외학술지에는 등재되지 않은 경우 
8. 해외학술지에는 등재되었지만 KCI에는 등재되지 않은 경우

In [11]:
data = pd.DataFrame(columns=['논문이름', '이름', '저자정보', 'KCI_등재_정보', '해외_학술지_등재_정보', 'KCI_IF', 'WOS_IF', 'test_info'])

# True Data 모든 항목을 정확히 기재한 경우 

'CASE 1' 
'https://www.kci.go.kr/kciportal/ci/sereArticleSearch/ciSereArtiView.kci?sereArticleSearchBean.artiId=ART001622576'
true_data = {'논문이름': 'Oxidative Stress Mediates Chemical Hypoxia-Induced Injury and Inflammation by Activating NF-κb-COX-2 Pathway in HaCaT Cells'
           , '이름': 'Chuntao Yang'
           , 'KCI_등재_정보': '등재'
           , '저자정보' : '1'
           , '해외_학술지_등재_정보': ['SCIE', 'SCI', 'SCOPUS']
           , 'KCI_IF': '0.47'
           , 'WOS_IF' :'2.38'
           , 'test_info' : 'True Data'
            }
data.loc[len(data)] = true_data

true_data = {'논문이름': 'Estimation of Genetic Parameters for Economic Traits of Hanwoo Cows Using Ultrasound'
           , '이름': '박경도'
           , 'KCI_등재_정보': '등재'
           , '저자정보' : '2'
           , '해외_학술지_등재_정보': ['SCIE', 'SCOPUS']
           , 'KCI_IF': '0.47'
           , 'WOS_IF' :'2.38'
           , 'test_info' : 'True Data'
            }
data.loc[len(data)] = true_data

# # False Data 
# # Case1 논문이 없는 경우 
# false_data = {'논문이름': 'Tintinnid Species as Biological Indicators for Monitoring Intrusion of the Warm Oceanic Waters into Korean Coastal Waters'
#            , '이름': '잘못된 이름'
#            , 'KCI_등재_정보': 'KCI 등재 후보'
#            , '저자정보' : '2'
#            , '해외_학술지_등재_정보': 'SCIE'
#            , 'KCI_IF': '0.22'
#            , 'WOS_IF' :'1.46'
#            , 'test_info' : '논문이 없는 경우'
#             } 
# data.loc[len(data)] = false_data

# # Case2 논문은 있지만 저자 정보가 잘못된 경우  
# false_data = {'논문이름': 'Tintinnid Species as Biological Indicators for Monitoring Intrusion of the Warm Oceanic Waters into Korean Coastal Waters'
#            , '이름': '잘못된 이름'
#            , 'KCI_등재_정보': 'KCI 등재 후보'
#            , '저자정보' : '2'
#            , '해외_학술지_등재_정보': 'SCIE'
#            , 'KCI_IF': '0.22'
#            , 'WOS_IF' :'1.46'
#            , 'test_info' : '저자 정보가 잘못된 경우'
#              } 
# data.loc[len(data)] = false_data

# # Case3 논문은 있지만 KCI 등재 정보가 잘못된 경우 
# false_data = {'논문이름': 'Tintinnid Species as Biological Indicators for Monitoring Intrusion of the Warm Oceanic Waters into Korean Coastal Waters'
#            , '이름': '잘못된 이름'
#            , 'KCI_등재_정보': 'KCI 등재 후보'
#            , '저자정보' : '2'
#            , '해외_학술지_등재_정보': 'SCIE'
#            , 'KCI_IF': '0.22'
#            , 'WOS_IF' :'1.46'
#            , 'test_info' : 'KCI 등재 정보가 잘못된 경우'
#              } 
# data.loc[len(data)] = false_data

# # Case4 논문은 있지만 해외 학술지 등재 정보가 잘못된 경우 
# false_data = {'논문이름': 'Tintinnid Species as Biological Indicators for Monitoring Intrusion of the Warm Oceanic Waters into Korean Coastal Waters'
#            , '이름': '잘못된 이름'
#            , 'KCI_등재_정보': 'KCI 등재 후보'
#            , '저자정보' : '2'
#            , '해외_학술지_등재_정보': 'SCIE'
#            , 'KCI_IF': '0.22'
#            , 'WOS_IF' :'1.46'
#            , 'test_info' : '해외 학술지 등재 정보가 잘못된 경우'
#              } 
# data.loc[len(data)] = false_data

# # Case5 논문은 있지만 IF 정보가 잘못된 것들 ( IF는 2022년 기준 최근 정보 기준으로 함 )
# false_data = {'논문이름': 'Tintinnid Species as Biological Indicators for Monitoring Intrusion of the Warm Oceanic Waters into Korean Coastal Waters'
#            , '이름': '잘못된 이름'
#            , 'KCI_등재_정보': 'KCI 등재 후보'
#            , '저자정보' : '2'
#            , '해외_학술지_등재_정보': 'SCIE'
#            , 'KCI_IF': '0.22'
#            , 'WOS_IF' :'1.46'
#            , 'test_info' : 'IF 정보가 잘못된 경우'
#              } 
# data.loc[len(data)] = false_data


# # Case6 KCI에는 등재되지 않았을 경우 
# false_data = {'논문이름': 'Tintinnid Species as Biological Indicators for Monitoring Intrusion of the Warm Oceanic Waters into Korean Coastal Waters'
#            , '이름': '잘못된 이름'
#            , 'KCI_등재_정보': '미등재'
#            , '저자정보' : '2'
#            , '해외_학술지_등재_정보': 'SCIE'
#            , 'KCI_IF': '0.22'
#            , 'WOS_IF' :'1.46'
#            , 'test_info' : 'KCI에 등재되지 않았을때'              
#         } 
# data.loc[len(data)] = false_data

# # Case6 KCI에는 등재되었지만, 해외 학술지에 등재되지 않았을 경우 
# false_data = {'논문이름': 'Tintinnid Species as Biological Indicators for Monitoring Intrusion of the Warm Oceanic Waters into Korean Coastal Waters'
#            , '이름': '잘못된 이름'
#            , 'KCI_등재_정보': 'KCI 등재'
#            , '저자정보' : '2'
#            , '해외_학술지_등재_정보': '등록 안된 case'
#            , 'KCI_IF': '0.22'
#            , 'WOS_IF' :'1.46'
#            , 'test_info' : 'KCI에 등재되었지만 해외학술지에 등재되지 않은 경우'              
#         } 
# data.loc[len(data)] = false_data

# # Case6 KCI에는 등재되었지만, 해외 학술지에 등재되지 않았을 경우 
# false_data = {'논문이름': 'Tintinnid Species as Biological Indicators for Monitoring Intrusion of the Warm Oceanic Waters into Korean Coastal Waters'
#            , '이름': '잘못된 이름'
#            , 'KCI_등재_정보': 'KCI 등재'
#            , '저자정보' : '2'
#            , '해외_학술지_등재_정보': '등록 안된 case'
#            , 'KCI_IF': '0.22'
#            , 'WOS_IF' :'1.46'
#            , 'test_info' : 'KCI에 미등재, 해외 학술지에는 등재 되었을 경우 '              
#         } 
# data.loc[len(data)] = false_data


In [13]:
for index, row in data.iterrows():

    print("Test Case Information : ", row['test_info'])
         
    article_id = get_article_id(row['논문이름'])
    if article_id is None:
        continue
    
    journal_id = article_validation(article_id, row['이름'], row['저자정보'])
    if journal_id is None:
        continue
    
    journal_validation(journal_id, row['KCI_등재_정보'],  row['해외_학술지_등재_정보'], row['KCI_IF'], row['WOS_IF'])
    
    print('---------------------------------------------------------------------------')

Test Case Information :  True Data
입력된 저자가 논문에 포함됩니다.
입력된 저자정보 : 1, 검색된 저자정보 : 1 로 저자 정보가 같습니다.
입력된 KCI 등재 정보 : 등재, 검색된 KCI 등재 정보 : KCI 등재 로  같지 않습니다.
등재된 해외 학술지 :  SCIE
입력된 해외학술지 : SCI 은 등재되어있지 않습니다.
등재된 해외 학술지 :  SCOPUS
입력된 KCI IF 정보가 불일치 합니다. 
입력된 WOF IF 정보가 불일치 합니다. 
---------------------------------------------------------------------------
Test Case Information :  True Data
입력된 저자가 논문에 포함됩니다.
입력된 저자정보 : 2, 검색된 저자정보 : 2 로 저자 정보가 같습니다.
KCI에 등록되지 않은 논문.
입력된 해외학술지 : SCIE 은 등재되어있지 않습니다.
입력된 해외학술지 : SCOPUS 은 등재되어있지 않습니다.
입력된 KCI IF 정보가 일치합니다.
입력된 WOF IF 정보가 일치합니다.
---------------------------------------------------------------------------


## API 호출 함수들

In [12]:
URL = "https://open.kci.go.kr/po/openapi/openApiSearch.kci"
API_KEY = "77670960"

def get_article_id(article_title):
    api_code = "articleSearch"
    search_keyword = article_title
    params = {
        'apiCode': api_code,
        'key': API_KEY,
        'title': search_keyword
    }
    response = requests.get(URL, params=params)

    if response.status_code == 200:
        contents = response.text
        xml_obj = bs4.BeautifulSoup(contents,'lxml-xml')
        # article-id 값을 추출
        article_id_tag = xml_obj.find('articleInfo', {'article-id': True})
        if article_id_tag:
            article_id = article_id_tag['article-id']
            return article_id
        else:
            print("{0} 논문을 찾을 수 없습니다.".format(article_title))
            return None
        
    else:
        print("요청 실패. 상태 코드:", response.status_code)
        return None
        
        
def article_validation(article_id, author, author_division):
    api_code = "articleDetail"
    params = {
        'apiCode': api_code,
        'key': API_KEY,
        'id': article_id
    }
    response = requests.get(URL, params=params)

    if response.status_code == 200:
        contents = response.text
        xml_obj = bs4.BeautifulSoup(contents,'lxml-xml')
        
        # 저자 정보 확인
        author_tags = xml_obj.find_all('author')
        is_author = False
        for author_tag in author_tags:
            if author_tag.find('name'):          
                name_text = author_tag.find('name').text
               
                if name_text == author:
                    is_author = True
                    print("입력된 저자가 논문에 포함됩니다.")
                    # 저자가 1저자인지 2저자인지
                    division = author_tag['author-division']
                    if author_division == division:
                        print("입력된 저자정보 : {0}, 검색된 저자정보 : {1} 로 저자 정보가 같습니다.".format(author_division, division))
                    else:
                        print("입력된 저자정보 : {0}, 검색된 저자정보 : {1} 로 저자 정보가 같지 않습니다.".format(author_division, division))
        if not is_author:
            print("입력된 저자가 논문에 포함되지 않습니다")
            # return None
        
        # 학회지ID값 검색
        journal_id_tag = xml_obj.find('journalInfo', {'journal-id': True})
        if journal_id_tag:
            journal_id = journal_id_tag['journal-id']
            # print("journal_id 값:", journal_id)
            return journal_id
        else:
            print("해당 논문이 작성된 학회가 존재하지 않습니다.")
            return None

    else:
        print("요청 실패. 상태 코드:", response.status_code)
        return None

    
              
def journal_validation(jounal_id, KCI_info, foreign_infos, KCI_IF, WOS_IF):
    api_code = "citationDetail"
    params = {
        'apiCode': api_code,
        'key': API_KEY,
        'id': jounal_id
    }
    response = requests.get(URL, params=params)
    if response.status_code == 200:
        contents = response.text
        xml_obj = bs4.BeautifulSoup(contents,'lxml-xml')
         
        kci_registration_tag = xml_obj.find('kci-registration')
        
        # KCI 등록 정보 확인 
        if kci_registration_tag:
            kci_registration_text = kci_registration_tag.text
            if KCI_info == kci_registration_text:
                print("입력된 KCI 등재 정보 : {0}, 검색된 KCI 등재 정보 : {1} 로  같지 않습니다.".format(KCI_info, kci_registration_text))
            else:
                print("입력된 KCI 등재 정보 : {0}, 검색된 KCI 등재 정보 : {1} 로  같지 않습니다.".format(KCI_info, kci_registration_text))
        else:
            print("KCI에 등록되지 않은 논문.")
            
        # 해외 학술지 정보 확인     
        foreign_registration_tags = xml_obj.find_all('foreign-registration')
        foreign_registration_tag_texts = [foreign_registration_tag.text for foreign_registration_tag in foreign_registration_tags if foreign_registration_tag]

        for foreign_info in foreign_infos:
            if foreign_info in foreign_registration_tag_texts:
                print('등재된 해외 학술지 : ', foreign_info)
            else:
                print('입력된 해외학술지 : {0} 은 등재되어있지 않습니다.'.format(foreign_info))
        
      
        # 이런식으로 하면 가장 최근의 2022년도의 IF를 갖고옴 
        # 표기되어있는 정보로는 2021, 2020 ~ 2008 여러 년도의 IF가 표기되어있음 
        IF_tags = xml_obj.find_all('impactFactor') 
        IF_tags_texts = [IF_tag.text for IF_tag in IF_tags if IF_tag]
        if KCI_IF in IF_tags_texts:
            print("입력된 KCI IF 정보가 일치합니다.")
        else:
            print("입력된 KCI IF 정보가 불일치 합니다. ")
       
        WOS_IF_tags = xml_obj.find_all('wosImpactFactor')
        WOS_IF_tags_texts = [WOS_IF_tag.text for WOS_IF_tag in WOS_IF_tags if WOS_IF_tag]
        if WOS_IF in WOS_IF_tags_texts:
            print("입력된 WOF IF 정보가 일치합니다.")
        else:
            print("입력된 WOF IF 정보가 불일치 합니다. ")
    return  None   
    
# test_id = 'ART001456060'    
# # journal_id = article_validation(test_id)
# journal_id = '000256'
# journal_validation(journal_id)
        

              
              
              

    
